In [1]:
# Play with the initial embedding matrix, the embedding values of tokens
# without inference through the model. These are useful for understanding
# the initial embeddings prior to context-sensitive learning and attention.
#
# This notebook supports the publication of James E. Dobson, "On Reading and 
# Interpreting Black Box Deep Neural Networks," International Journal
# of Digital Humanities (2023). https://doi.org/10.1007/s42803-023-00075-w
#
# James E. Dobson
# Dartmouth College
# https://jeddobson.github.io/

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# load GPT2
model = AutoModelForCausalLM.from_pretrained("gpt2",
                                             output_hidden_states=True,
                                             low_cpu_mem_usage=True)
tok = AutoTokenizer.from_pretrained("gpt2")

# end of sentence/text token padding
tok.pad_token = tok.eos_token

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [3]:
def describe_model():
    config = model.config.__dict__
    print("Model type: {0} ({1})".format(config['model_type'],
                                         ' '.join(config['architectures'])))
    print("Vocab size: {0}".format(config['vocab_size']))
    print("Layers: {0}".format(config['n_layer']))
    print("Embedding width: {0}".format(config['n_embd']))
    print("Parameters:\n Output Attentions: {0}\n Output Hidden States: {1}"
          .format(config['output_attentions'],
                 config['output_hidden_states']))

In [4]:
describe_model()

Model type: gpt2 (GPT2LMHeadModel)
Vocab size: 50257
Layers: 12
Embedding width: 768
Parameters:
 Output Attentions: False
 Output Hidden States: True


In [5]:
# initial embedding matrix
# model.transformer.wte.weight or model.lm_head.weight
token_embedding_matrix = model.transformer.wte.weight

In [6]:
# This is a simple strategy to measure semantic similarity (distance)
# using cosine similarity from the intitial embedding matrix. Because
# we are using the mean embedding, this can also create embeddings for longer
# fragments, but there is no attention mechanism used. This means that token
# similarity (float, floatation) is the rule.
# 
# We're using inverse similarity, so distance will range from 0 (close)
# to 1 (distant).

def token_similarity(word1,word2):
    embs = []
    for word in word1, word2:
        inp_tok = tok(word,
             padding=True,
             return_tensors="pt").to(next(model.parameters()).device)
        input_ids = inp_tok["input_ids"]
        mean_embedding = torch.mean(token_embedding_matrix[input_ids],dim=1)
        embs.append(mean_embedding.detach().numpy())
    dist = 1 - cosine_similarity(embs[0],embs[1])[0][0]
    return dist

In [7]:
token_similarity("float","buoyant")

0.662583202123642

In [8]:
token_similarity("float","string")

0.5526950657367706

In [9]:
token_similarity("float","floatation")

0.21155625581741333

In [10]:
token_similarity("blue","yellow")

0.4116727113723755

In [11]:
token_similarity("desire","desired")

0.19898557662963867